# Class Inheritance

Class inheritance allows a class to "borrow" properties from another class.

By default, the class will inherit all the properties from it's parent / base class.

The class can then overwrite the inherited properties and / or add properties of its own.

In [ ]:
class BaseClass:
    """Base Class."""

    name = "BaseClass"
    x = 1
    
    
# Note the BaseClass between brackets!
class Subclass(BaseClass):
    """I'm a subclass of BaseClass."""
    
    # Only overwrite the name property
    name = "Subclass"

In [ ]:
# Create instance of the base class
base = BaseClass()
base.name

In [ ]:
# As expected it has x = 1
base.x

In [ ]:
# Create an instance of the BaseSubclass
sub = Subclass()
sub.name

In [ ]:
# Note how BaseSubclass inherits x = 1 from BaseClass
sub.x

## A More Practical Example

Imagine a project with 2 sensors that both log data in delimited text files. While the file format is similar, there are subtle differences in separators, date format used, et cetera


Sensor A logs in this format:

```
TIME;TEMP;HUM
2020-01-01 14:00:00;20.1;40.0
2020-01-01 14:00:01;20.2;39.8
2020-01-01 14:00:02;20.3;40.0
2020-01-01 14:00:03;20.3;40.2
2020-01-01 14:00:04;20.3;40.0
```

Sensor B logs in this format:

```
TS|CO2|NO2
1577887200|602.200|1.973
1577887260|599.917|2.270
1577887320|598.083|2.842
1577887380|596.600|2.590
1577887440|599.083|2.692
```

Note the difference in time notation and use of a different delimiter character.

### The BaseReader Class

In [ ]:
import pandas as pd


class BaseReader:
    """Class for reading delimited text files"""

    # Set delimited file parameters
    # Note: see pd.read_csv for available options
    _csv_params = {}
    
    # Set required columns
    _required = []
    
    # Set renaming options
    # Note: Supply either a dict or callable
    _rename = None
        
    def _load_file(self, path):
        """Load delimited file with specified parameters."""
        
        return pd.read_csv(path, **self._csv_params)

    def _check_required(self, df):
        """Checks required columns are present."""
        
        missing = set(self._required) - set(df.columns)
        if missing:
            raise RuntimeError(
                f"Missing columns in the data: {', '.join(missing)}"
            )

    def _rename_columns(self, df):
        """Renames columns if requested."""
        
        if isinstance(self._rename, (dict, callable)):
            return df.rename(columns=self._rename)
        
    def post_process(self, df):
        """Processes the data after loading it."""
        
        return df
            
    def read(self, path):
        """Reads delimitied data from the specified path."""
        
        df = self._load_file(path)
        self._check_required(df)
        
        # Process the data
        return (
            df
            .pipe(self._rename_columns)
            .pipe(self.post_process)
        )

### Sensor A Reader

In [ ]:
class SensorAReader(BaseReader):
    """Class for reading sensor A data files."""

    _csv_params = {"sep": ";", "parse_dates": ["TIME"]}
    _required = ["TIME", "TEMP", "HUM"]
    _rename = {"TIME": "datetime", "TEMP": "temperature", "HUM": "humidity_pct"}

In [ ]:
reader_a = SensorAReader()
reader_a.read("data/sensor_a/20200101.dat").head()

### Sensor B Reader

In [ ]:
class SensorBReader(BaseReader):
    """Class for reading sensor B data files."""
    
    _csv_params = {"sep": "|"}
    _required = ["TS", "CO2", "NO2"]
    _rename = {"TS": "datetime"}
    
    def post_process(self, df):
        """Convert epoch time to datetime."""

        return df.assign(datetime=lambda df: pd.to_datetime(df["datetime"], unit="s"))

In [ ]:
reader_b = SensorBReader()
reader_b.read("data/sensor_b/20200101.dat").head()

## Inheritance and Object Types

In [ ]:
# Type refers to the specific instance type
type(reader_a)

In [ ]:
# Can use isinstance for the subclass
isinstance(reader_a, SensorAReader)

In [ ]:
# Obviously False for different subclass
isinstance(reader_a, SensorBReader)

In [ ]:
# But also matches the base class
isinstance(reader_a, BaseReader)

In [ ]:
# Explicit check on subclass
issubclass(SensorAReader, BaseReader)

## Accessing the Parent Class

In [ ]:
class BaseClass:
    """Base class."""

    def __init__(self):
        print("Called __init__ from BaseClass.")


class Subclass(BaseClass):
    """Subclass extending BaseClass."""
    
    def __init__(self):
        
        # Use super() to access the base class
        super().__init__()

        print("Called __init__ from Subclass")

In [ ]:
sub = Subclass()

## Accessing Subclasses

Classes can access thier subclasses through the `__subclasses__()` method

In [ ]:
class BaseReader:
    """Data reader base class."""
    
    @classmethod
    def available_readers(cls):
        return [cls.__name__ for cls in cls.__subclasses__()]

class ReaderA(BaseReader):
    """Subclass A extending BaseReader."""

class ReaderB(BaseReader):
    """Subclass B extending BaseReader."""


In [ ]:
BaseReader.available_readers()

## More Complex Inheritance

### Depth: Specialization

No limits on the "depth" of the inheritance structure; you can create ever more specialized classes.

In [ ]:
class Animal:
    kingdom = "Animals"
    
class Vertebrate(Animal):
    phylum = "Chordata"

class Mammal(Vertebrate):
    class_ = "Mammals"

class Primate(Mammal):
    order = "Primates"
    
class HumanLike(Primate):
    family = "Hominids"
    
class ModernHuman(HumanLike):
    genus = "Homo"
    
class Human(ModernHuman):
    species = "Homo Sapiens"

In [ ]:
for attr in "kingdom", "phylum", "class_", "order", "family", "genus", "species":
    print(f"{attr} = {getattr(Human, attr)}")

### Width: Adaptation

Classes can inherit from more than 1 class at the same time. The properties of all base classes are combined:

In [ ]:
class BaseEstimator:
    
    def fit(self, X, y=None):
        """Fit method stub."""
        
        pass
    
    def predict(self, X):
        """Predict method stub"""


class BoostingMixin:
    
    def subsample(self, X, y, nsamples):
        """Create subsamples from the data"""
        
        pass
    

class MyBoost(BaseEstimator, BoostingMixin):
    """Model class using mix-ins to extend functionality."""
    
    pass

In [ ]:
[_ for _ in dir(MyBoost) if not _.startswith("__")]

### Order of inheritance matters!

In [ ]:
class BaseA:
    """Base class A implementing a test method."""
    
    def test(self):
        print("Hi, I'm from BaseA!")
        
class BaseB:
    """Base class B also implementing a test method."""
    
    def test(self):
        print("Hi, I'm from BaseB!")

In [ ]:
class SubOne(BaseA, BaseB):
    """Subclass inheriting from A and B."""

In [ ]:
# Seems left-most base class wins!
SubOne().test()

In [ ]:
class SubTwo(BaseB, BaseA):
    """Subclass inheriting from B and A."""

In [ ]:
SubTwo().test()

## Protected Attributes

In [ ]:
class BaseClass:
    """Base class."""
    
    normal = "base"
    __protected = "base"

    def get_attributes(self):
        return f"normal={self.normal}, protected={self.__protected}"
    
class Subclass(BaseClass):
    """Subclass extending BaseClass."""

    normal = "subclass"
    __protected = "subclass"

    def get_attributes(self):
        return f"normal={self.normal}, protected={self.__protected}"
    
    def print_types(self):
        print("From super: ", super().get_attributes())
        print("From self:  ", self.get_attributes())

In [ ]:
sub = Subclass()
sub.print_types()

In [ ]:
[_ for _ in dir(sub) if not _.startswith("__")]

## Abstract Base Classes

In [ ]:
class Polygon:
    
    def num_sides(self):
        raise NotImplementedError("All polygons must define a num_sides method!")

In [ ]:
class Triangle(Shape):
    pass

In [ ]:
Triangle().num_sides()

In [ ]:
from abc import ABC, abstractmethod


class Polygon(ABC):
    
    @abstractmethod
    def num_sides(self):
        pass
    
class Triangle(Polygon):
    pass

In [ ]:
Polygon()

In [ ]:
Triangle()